In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sastrawi

     |████████████████████████████████| 209 kB 5.2 MB/s 


In [3]:
import pandas as pd
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [51]:
df = pd.read_excel("/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/data latih.xlsx")
df.head()

,Tweet,Class
0,Hari ini udh mulai ppkm yaa..😣,2
1,@e100ss mohon info apakah PGS pasar turi selam...,2
2,@bertanyarl Di rumah aja soalnya lagi ppkm\n\n...,2
3,Pangkal penanganan pandemi #covid19 di Indones...,1
4,ppkm mikro anjingggggggg,1


In [52]:
df_test = pd.read_excel("/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/data uji.xlsx")
df_test.head()

,ID,Tweet,Class
0,1,Ppkm buat kami yg ga dipulau jawapun rada deg ...,?
1,2,"Ditengah orang2 yang pada mikir ;\n- ""waduh PP...",?
2,3,PPKM KENAPA BERLAKU SAMPAI 20 JULI..!? TANGGAL...,?
3,4,PPKM kenapa sumatera ngga ya? Dan lain2 juga. ...,?
4,5,sch! ppkm darurat kan cuman jawa-bali ya berar...,?


In [53]:
KOLOM_LABEL= "Class" #@param {type:"string"}
print(df[KOLOM_LABEL].unique())
print(df_test[KOLOM_LABEL].unique())


[2 1 3]
['?']


In [54]:
class DataCleaning:
  # Initialization
  factory     = StemmerFactory()
  stemmer     = factory.create_stemmer()
  kamus_alay1 = pd.read_csv('https://raw.githubusercontent.com/fendiirfan/Kamus-Alay/main/Kamu-Alay.csv')
  kamus_alay1 = kamus_alay1.set_index('kataAlay')
  kamus_alay2 = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
  kamus_alay2 = kamus_alay2.filter(['slang', 'formal'], axis=1)
  kamus_alay2 = kamus_alay2.drop_duplicates(subset=['slang'], keep='first')
  kamus_alay2 = kamus_alay2.set_index('slang')
  stopword1   = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
  custom_word = []

  @classmethod
  def CleanDataFrame(cls, df, col_name, label_name, jum_minimum=None, minimum_kata=0, label_mapping=None, dropna=False):

    final_list_clean = []
    final_list_kotor = []
    final_label = []

    if jum_minimum == None: jum_minimum = len(df)
    if len(df) < jum_minimum: raise "Jumlah Data Yang Diinginkan melebihi Data yang Ada"
    i = 0
    current = 0
    
    while i < len(df):
      current_kalimat = df.loc[i][col_name]
      current_label = df.loc[i][label_name]

      clean_kalimat = cls.__cleanSentence__(current_kalimat)
      if type(clean_kalimat) != str or clean_kalimat == None or clean_kalimat == "":
        print("Ditemukan string kosong setelah di preprocessed pada kalimat: ", current_kalimat)
        clean_kalimat = "NaN"
      if (len(clean_kalimat.split(' ')) >= minimum_kata):
        final_list_clean.append(str(clean_kalimat))
        final_list_kotor.append(str(current_kalimat))
        if label_mapping != None:
          final_label.append(label_mapping[current_label])
        else:
          final_label.append(current_label)
        current += 1

        if current % 10 == 0:
          print("Memproses {} data".format(current))

      if current == jum_minimum:
        break

      i += 1
    
    data = {
        'raw': final_list_kotor,
        'processed': final_list_clean,
        'label': final_label
    }

    final_df = pd.DataFrame(data)
    if dropna:
      print("NaN Dropped")
      final_df = final_df.dropna(how='any')

    final_df['processed'] = final_df['processed'].astype(str)
    final_df['raw'] = final_df['raw'].astype(str)
    return final_df

  @classmethod
  def __cleanSentence__(cls, text):
    '''
    Melakukan prapemrosesan pada suatu kalimat dengan menghilangkan formatting pada kalimat,
    menghilangkan stopword pada kalimat, mengganti kata alay yang sudah terdefinisikan, serta
    melakukan stemming kalimat tersebut.
    '''

    # #
    # Cleaning Formatted Text Link And Tag (@username) using Regex
    # #
    text = re.sub(r'http\S+', '', text)
    text = re.sub('(@\w+|#\w+)','',text)

    #will replace the html characters with " "
    text = re.sub('<.*?>', '', text)  

    temp_text = list(text)
    for i in range(len(temp_text)):
      if temp_text[i] in string.punctuation:
        temp_text[i] = " "

    text = ''.join(temp_text)

    #will consider only alphabets
    text = re.sub('[^a-zA-Z]',' ',text) 
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will replace a word
    text = re.sub("(username|user|url|rt|xf|fx|xe|xa)\s|\s(user|url|rt|xf|fx|xe|xa)","",text)
    # will repalce repated char
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)
    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)
    # will replace space more than one
    text = re.sub('(s{2,})',' ',text)
    # will join the words
    text=' '.join(text.split())

    text_split = text.split(' ')
    # #
    # Mengganti kata-kata yang tidak baku
    # aku gapakai try catch lagi, lebih simple malah ini
    # #
    for i in range(len(text_split)):
      if text_split[i] in cls.kamus_alay1.index:
        text_split[i] = cls.kamus_alay1.loc[text_split[i]]['kataBaik']
      elif text_split[i] in cls.kamus_alay2.index:
        text_split[i] = cls.kamus_alay2.loc[text_split[i]]['formal']

    # #
    # Stemming
    # #
    stemmed_text = cls.stemmer.stem(text)

    # #
    # Removing Stopwords and custom word
    # #
    temp_text_split = []
    for i in range(len(text_split)):
      if (type(text_split[i]) == str):
        temp_text_split.append(text_split[i])

    # Memastikan saja
    if len(temp_text_split) == 0:
      return ""
    else:
      final_text = ' '.join(temp_text_split)
    
    return final_text

In [55]:
label_mapping = {
    1  : 0,
    2  : 1,
    3  : 2,
}

# dropna=True hanya untuk training
df = DataCleaning.CleanDataFrame(df, 'Tweet', 'Class', None, 2, label_mapping, dropna=True)
print(len(df))

Memproses 10 data
Memproses 20 data
Memproses 30 data
Memproses 40 data
Memproses 50 data
Memproses 60 data
Memproses 70 data
Memproses 80 data
Memproses 90 data
Memproses 100 data
Memproses 110 data
Memproses 120 data
Memproses 130 data
Memproses 140 data
Memproses 150 data
Memproses 160 data
Memproses 170 data
Memproses 180 data
Memproses 190 data
Memproses 200 data
Memproses 210 data
Memproses 220 data
Memproses 230 data
Memproses 240 data
Memproses 250 data
Memproses 260 data
Memproses 270 data
Memproses 280 data
Memproses 290 data
Memproses 300 data
Memproses 310 data
Memproses 320 data
Memproses 330 data
Memproses 340 data
Memproses 350 data
Memproses 360 data
Memproses 370 data
Memproses 380 data
Memproses 390 data
Memproses 400 data
Memproses 410 data
Memproses 420 data
Memproses 430 data
Memproses 440 data
Memproses 450 data
Memproses 460 data
Memproses 470 data
Memproses 480 data
Memproses 490 data
Memproses 500 data
Memproses 510 data
Memproses 520 data
Memproses 530 data
Me

In [58]:
# Jangan Ditimpa Data Aslinya
df.to_csv('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Unbalance/NonStopword_Stem_Min2_Clean/clean_train.csv', index=False, encoding="", na_rep="")
df[df['processed'] == "NaN"]

,raw,processed,label


In [60]:
df_test = DataCleaning.CleanDataFrame(df_test, 'Tweet', 'Class', None, 0, None, dropna=True)

# Jangan Ditimpa Data Aslinya
df_test.to_csv('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Unbalance/NonStopword_Stem_Min2_Clean/clean_test.csv', index=False)

Memproses 10 data
Memproses 20 data
Memproses 30 data
Memproses 40 data
Memproses 50 data
Memproses 60 data
Memproses 70 data
Memproses 80 data
Memproses 90 data
Memproses 100 data
Memproses 110 data
Memproses 120 data
Memproses 130 data
Memproses 140 data
Memproses 150 data
Memproses 160 data
Memproses 170 data
Memproses 180 data
Memproses 190 data
Memproses 200 data
NaN Dropped
